# Llama 2 - Setup

Llama ist ein offenes Sprachmodell, welches von Meta entwickelt und trainiert wurde.
Auf Basis des Vorgängers, der wohl unabsichtlicht im Internet aufgetaucht ist, wurden nicht nur von Wissenschaffenden leistungsfähige Derivate entwickelt.

### Konfiguration

Der GitHub-Nutzer `oobabooga` hat durch seine webbasierte Nutzungsschnittstelle die Einrichtung und den Betrieb von LLama-Modellen stark vereinfacht. In dem hier vorgestellten Arbeitsablauf, muss nur der Installationsort festgelegt werden:

In [ ]:
from pathlib import Path

# Where should the webui be downloaded to?
LLAMA_ROOT = Path("../webui")

# This is only relevant for remote hosting
# GRADIO_USER = __user__
# GRADIO_PASS = __password__
# !rm -rf {LLAMA_ROOT} && mkdir -p {LLAMA_ROOT}

### Plattformspezifische Anpassungen

Im besten Fall werden die restlichen Parameter automatisch gesetzt. Sollte das jedoch nicht funktionieren, können Nutzende die Umgebungsvariablen für den folgenden Installer auch händisch setzen.

In [ ]:
import platform
import subprocess

if platform.system() == "Darwin":
    OS = "macos"
elif platform.system() == "Windows":
    OS = "windows"
else:
    OS = "linux"

INSTALLER = LLAMA_ROOT.joinpath(f"start_{OS}.{'bat' if OS == 'windows' else 'sh'}")

try:
    subprocess.check_output('nvidia-smi')
    GPU_CHOICE = "A"  # use CUDA
except Exception:
    if platform.machine() == "arm":
        GPU_CHOICE = "C"  # use MPS for M generation Macs
    else:
        GPU_CHOICE = "N"  # use CPU only

### Text Generation Web UI herunterladen

Es ist wieder Zeit ein Projekt zu 'klonen'!

In [ ]:
!git clone https://github.com/oobabooga/text-generation-webui.git {LLAMA_ROOT}

### Text Generation Web UI starten

Wir nutzen den 'One-Click-Installer', den `oobabooga`.
Dieser lädt `conda` zwar erneut herunter, was aber in Anbetracht der Größe der herunterzuladenden Modelle nicht weiter ins Gewicht fällt.

In [ ]:
%env LLAMA_ROOT {LLAMA_ROOT}
%env GPU_CHOICE {GPU_CHOICE}
# If you know you have an AMD GPU and are on Linux or MacOS, you can set 
# GPU_CHOICE manually to "B" or "D" if you got an Intel Arc (IPEX) GPU
# %env GPU_CHOICE = "B"
%env USE_CUDA118 "N"  # if you have issues set this one to "Y"
%env INSTALLER {INSTALLER}
# This is only needed for remote hosting
# %env COMMANDLINE_ARGS --share --gradio-auth {GRADIO_USER}:{GRADIO_PASS}

!cd ${LLAMA_ROOT} && ${INSTALLER} ${COMMANDLINE_ARGS}

### Wir brauchen Modelle

Die Web UI erlaubt es, Sprachmodelle direkt aus der Weboberfläche von [Hugging Face](https://huggingface.co) (HF) herunterzuladen.
Dazu muss im Reiter `Model` nur der sogenannte HF-Handle eingegeben werden.
Sollte der Handle nicht eindeutig sein, kann noch die spezifische Modeldatei angegeben werden.

Die Auswahl an Sprachmodellen ist unüberschauber und neu trainierte Modelle werden täglich veröffentlicht.
Eine Orientierung bietet das [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) von Hugging Face, welches Sprachmodelle und deren Leistungsfähigkeit in Benchark-Tests gegenüberstellt

|Handle|File|
|------|----|
|[Intel/neural-chat-7b-v3-1](https://huggingface.co/Intel/neural-chat-7b-v3-1)|---|
|[LeoLM/leo-hessianai-13b](https://huggingface.co/LeoLM/leo-hessianai-13b)|---|
|[TheBloke/leo-hessianai-7B-GGUF](https://huggingface.co/TheBloke/leo-hessianai-7B-GGUF)|leo-hessianai-7b.Q5_K_M.gguf|